In [28]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from lib.Utility import exportExcelWithTimeStamp

from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import numpy as np # Fundamental package for scientific computing with Python
import joblib
import plotly.express as px
from plotly import graph_objects as go
# Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [29]:
# data = pd.read_excel('serieA-2023-24.xlsx')
# data1920 = pd.read_excel('data/engdata/19-20.xlsx', index_col='Date')
# data2021 = pd.read_excel('data/engdata/20-21.xlsx', index_col='Date')
# data2122 = pd.read_excel('data/engdata/21-22.xlsx', index_col='Date')
# data2223 = pd.read_excel('data/engdata/22-23.xlsx', index_col='Date')
# data2324 = pd.read_excel('data/engdata/23-24.xlsx', index_col='Date')

folder = 'PremierLeague'

data1920 = pd.read_excel('data/engdata/'+ folder + '/19-20-E.xlsx', index_col='Date')
data2021 = pd.read_excel('data/engdata/'+ folder + '/20-21-E.xlsx', index_col='Date')
data2122 = pd.read_excel('data/engdata/'+ folder + '/21-22-E.xlsx', index_col='Date')
data2223 = pd.read_excel('data/engdata/'+ folder + '/22-23-E.xlsx', index_col='Date')
data2324 = pd.read_excel('data/engdata/'+ folder + '/23-24-E.xlsx', index_col='Date')

data = data1920

data = pd.concat([data, data2021, data2122, data2223, data2324])
print(data)
# data['Cluster'] = np.where(data['FTR'] == 'H', 1, np.where(data['FTR'] == 'A', 2, 0))

data.tail()

           Div      Time          HomeTeam          AwayTeam  FTHG  FTAG FTR  \
Date                                                                           
2019-08-09  E0  20:00:00         Liverpool           Norwich     4     1   H   
2019-08-10  E0  12:30:00          West Ham          Man City     0     5   A   
2019-08-10  E0  15:00:00       Bournemouth  Sheffield United     1     1   D   
2019-08-10  E0  15:00:00           Burnley       Southampton     3     0   H   
2019-08-10  E0  15:00:00    Crystal Palace           Everton     0     0   D   
...         ..       ...               ...               ...   ...   ...  ..   
2024-05-19  E0  16:00:00    Crystal Palace       Aston Villa     5     0   H   
2024-05-19  E0  16:00:00         Liverpool            Wolves     2     0   H   
2024-05-19  E0  16:00:00             Luton            Fulham     2     4   A   
2024-05-19  E0  16:00:00          Man City          West Ham     3     1   H   
2024-05-19  E0  16:00:00  Sheffield Unit

,Div,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,GoalsDifference,ConcededGoalsDifference,HomePoints,AwayPoints,HomeForm,AwayForm,HomeRecentHomeForm,AwayRecentAwayForm,HomeEloRating,AwayEloRating
Date,,,,,,,,,,,,,,,,,,,,,
2024-05-19,E0,16:00:00,Crystal Palace,Aston Villa,5,0,H,2,0,H,...,24,2,3,0,12.0,4.0,12.0,4.0,1504.204051,1551.471151
2024-05-19,E0,16:00:00,Liverpool,Wolves,2,0,H,2,0,H,...,34,22,3,0,12.0,2.0,12.0,2.0,1617.795533,1457.244866
2024-05-19,E0,16:00:00,Luton,Fulham,2,4,A,1,2,A,...,1,22,0,3,5.0,8.0,5.0,8.0,1380.462487,1477.011298
2024-05-19,E0,16:00:00,Man City,West Ham,3,1,H,2,1,H,...,34,38,3,0,13.0,3.0,13.0,3.0,1669.523665,1489.396699
2024-05-19,E0,16:00:00,Sheffield United,Tottenham,0,3,A,0,1,A,...,36,40,0,3,2.0,4.0,2.0,4.0,1336.910858,1537.147578


In [30]:
# Sostituire i valori infiniti con 100
data.replace([np.inf, -np.inf], 10, inplace=True)
# Sostituire i NaN (celle vuote) con 0 nei rapporti
data['HomeGoalsRatio'].fillna(0, inplace=True)
data['AwayGoalsRatio'].fillna(0, inplace=True)

resultFileName = 'data/mergedData'+ folder + '.xlsx'
data.to_excel(resultFileName)

C:\Users\vitto\AppData\Local\Temp\ipykernel_33132\4159932336.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['HomeGoalsRatio'].fillna(0, inplace=True)
C:\Users\vitto\AppData\Local\Temp\ipykernel_33132\4159932336.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp